<a href="https://colab.research.google.com/github/mshsu/stat400-sets/blob/main/crashes_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
# Query the Cal Poly police logs site for HTML source
response = requests.get("https://afd.calpoly.edu/police/campus-reports/logs")
# Create HTML parser
soup = BeautifulSoup(response.content, "html.parser")
# The data is found in an element denoted by <div class="row align-justify"><\div>
# Within that, it is found in a <table><\table> element
div = soup.find("div", {"class": "row align-justify"})
table = div.find("table")

# Initialize list of data row
rows = []
# Iterate through each row of the table to convert from HTML to dict
# Then dd to rows list
for tr in table.find_all("tr"):
  info = tr.find_all("td")
  if len(info) >= 5:
    # The `Details` column has multiple parts of information
    # This code splits up the `Details` column into other appropriate columns
    details = str(info[3]).replace("<td>", "").replace("</strong>", "").replace("<br/>", "").replace("</td", "").split("<strong>")[1:]
    dets = {"Incident": '', "CAD #": '', "Report #": ''}
    for detail in details:
      detail = detail.split(": ")
      dets[detail[0]] = detail[1]
    # Add info to row dictionary, then add to list
    rows.append({
        "Date/Time Reported": info[0].text,
        "Date/Time Occurred": info[1].text,
        "Location": info[2].text,
        "Incident": dets['Incident'],
        "CAD #": dets['CAD #'].replace(">", ""),
        "Report #": dets['Report #'].replace(">", ""),
        "Disposition": info[4].text
    })

# Turn rows list into tabular dataframe
Blotter = pd.DataFrame(rows)
# Export to CSV file
Blotter.to_csv("Blotter.csv")
# Print here
Blotter

,Date/Time Reported,Date/Time Occurred,Location,Incident,CAD #,Report #,Disposition
0,04/15/2023 12:02 pm,,Robert A. Mott Physical Education,PATROL ACTIVITY - Foot Patrol,23-04-15-005194,,
1,04/15/2023 11:26 am,,Yak?ityutyu - E,ALARM - Fire Alarm,23-04-15-005193,,TURNED OVER TO FACILITY SVS
2,04/15/2023 10:46 am,,Mathematics And Science,CITIZEN ASSIST - Public Assist (specify),23-04-15-005191,,TURNED OVER TO FACILITY SVS
3,04/15/2023 10:11 am,,Unknown On-campus,PATROL ACTIVITY - Patrol Check,23-04-15-005190,,IN SERVICE
4,04/15/2023 10:07 am,,,DISPATCH - 911 Unknown/Abandoned or Hangup,23-04-15-005189,,False/Human Error
...,...,...,...,...,...,...,...
2011,02/13/2023 07:11 pm,,Recreation Center,CITIZEN ASSIST - Escort,23-02-13-002066,,COMPLETED- NO REPORT
2012,02/13/2023 02:45 pm,,Pcv Huasna,ALARM - Elevator Emergency/Alarm,23-02-13-002052,,False/Human Error
2013,02/13/2023 01:58 pm,,Ytt Parking Structure,PARKING - Parking Violation,23-02-13-002051,,IN SERVICE
2014,02/13/2023 01:30 pm,,Sequoia Hall,FOLLOWUP - Followup,23-02-13-002050,,IN SERVICE
